# Eläinasiantuntijajärjestelmän toteuttaminen

Esimerkki [AI for Beginners Curriculum](http://github.com/microsoft/ai-for-beginners) -materiaalista.

Tässä esimerkissä toteutamme yksinkertaisen tietopohjaisen järjestelmän eläimen tunnistamiseksi fyysisten ominaisuuksien perusteella. Järjestelmä voidaan esittää seuraavana JA-TAI-puun osana (tämä on osa koko puuta, ja siihen voidaan helposti lisätä lisää sääntöjä):

![](../../../../../../translated_images/fi/AND-OR-Tree.5592d2c70187f283.webp)


## Oma asiantuntijajärjestelmämme kuori takaapäin päättelyllä

Yritetään määritellä yksinkertainen kieli tietämyksen esittämiseen tuotantosääntöjen pohjalta. Käytämme Python-luokkia avainsanoina sääntöjen määrittämiseen. Periaatteessa luokkia on kolmenlaisia:
* `Ask` edustaa kysymystä, joka täytyy kysyä käyttäjältä. Se sisältää joukon mahdollisia vastauksia.
* `If` edustaa sääntöä, ja se on vain syntaktinen sokeri säännön sisällön tallentamiseen
* `AND`/`OR` ovat luokkia, jotka edustavat JA/TAI-haaroja puussa. Ne vain tallentavat argumenttilistan sisälleen. Koodin yksinkertaistamiseksi kaikki toiminnallisuus on määritelty yläluokassa `Content`


In [1]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices),flush=True)
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

Järjestelmässämme työmuisti sisältäisi luettelon **tiedoista** **ominaisuus-arvopareina**. Tietopohja voidaan määritellä yhtenä suurena sanakirjana, joka yhdistää toiminnot (uudet tiedot, jotka tulisi lisätä työmuistiin) ehtoihin, ilmaistuna JA-TAI lausekkeina. Lisäksi joitakin tietoja voidaan `Kysyä`.


In [2]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

Taaksepäin päättelyn suorittamiseksi määrittelemme `Knowledgebase`-luokan. Se sisältää:
* Työmuistin `memory` - sanakirja, joka yhdistää attribuutit arvoihin
* Tietokannan `rules` yllä määritellyssä formaatissa

Kaksi päämenetelmää ovat:
* `get`, jolla saadaan attribuutin arvo, suorittaen päättely tarvittaessa. Esimerkiksi `get('color')` saa väri-kohdan arvon (se kysyy tarvittaessa ja tallentaa arvon myöhempää käyttöä varten työmuistiin). Jos kysymme `get('color:blue')`, se kysyy väriä ja palauttaa sitten arvon `y`/`n` riippuen väristä.
* `eval` suorittaa varsinaisen päättelyn, eli käy läpi JA/TAI-puun, arvioi alakohtia jne.


In [3]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

Määritellään nyt eläintietokantamme ja suoritetaan konsultaatio. Huomaa, että tämä kutsu esittää sinulle kysymyksiä. Vastata voit kirjoittamalla `y`/`n` kyllä/ei -kysymyksiin tai valitsemalla numeron (0..N) pidempiin monivalintakysymyksiin.


In [4]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n
sharp teeth
y/n
claws
y/n
forward-looking eyes
y/n
color
0. red-brown
1. black and white
2. other
has hooves
y/n
long neck
y/n
long legs
y/n
pattern
0. dark stripes
1. dark spots


'giraffe'

## Ekspertan käyttäminen eteenpäin päin päättelyyn

Seuraavassa esimerkissä yritämme toteuttaa eteenpäin päin päättelyn käyttäen yhtä tiedon esityksen kirjastoista, [Experta](https://github.com/nilp0inter/experta). **Experta** on kirjasto eteenpäin päin päättelyjärjestelmien luomiseen Pythonilla, joka on suunniteltu muistuttamaan klassista vanhaa järjestelmää [CLIPS](http://www.clipsrules.net/index.html).

Voisimme myös toteuttaa eteenpäin sarjoittamisen itse ilman suurempia ongelmia, mutta naiivit toteutukset eivät yleensä ole kovin tehokkaita. Tehokkaampaan sääntöjen sovitukseen käytetään erityistä algoritmia [Rete](https://en.wikipedia.org/wiki/Rete_algorithm).


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/nilp0inter/experta

  Cloning https://github.com/nilp0inter/experta to /tmp/pip-req-build-7qurtwk3
  Running command git clone --filter=blob:none --quiet https://github.com/nilp0inter/experta /tmp/pip-req-build-7qurtwk3
  Resolved https://github.com/nilp0inter/experta to commit c6d5834b123861f5ae09e7d07027dc98bec58741
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for experta: filename=experta-1.9.5.dev1-py3-none-any.whl size=34804 sha256=888c459512a5e713f4b674caa9a0f96cfdf07ec0d6eb56cc318ce0653d218014
  Stored in directory: /tmp/pip-ephem-wheel-cache-1eeii9zy/wheels/3d/e8/bb/22d7956359603fa8dd679aa09f5b8efb3f29991c3986fdc787
Successfully built experta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [experta]


In [13]:
from experta import *
#import experta

Määrittelemme järjestelmämme luokkana, joka perii `KnowledgeEngine`-luokan. Jokainen sääntö määritellään erillisenä funktiona, jossa on `@Rule`-annotaatio, joka määrittää, milloin sääntö aktivoituu. Säännön sisällä voimme lisätä uusia tietoja käyttämällä `declare`-funktiota, ja näiden tietojen lisääminen johtaa siihen, että eteenpäin päättelymoottori kutsuu lisää sääntöjä.


In [14]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sharp teeth'),Fact('claws'),Fact('forward looking eyes')),
           Fact('eats meat')))
    def cornivor(self):
        self.declare(Fact('carnivor'))
        
    @Rule(OR(Fact('hair'),Fact('gives milk')))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('mammal'),
          OR(Fact('has hooves'),Fact('chews cud')))
    def hooves(self):
        self.declare('ungulate')
        
    @Rule(OR(Fact('feathers'),AND(Fact('flies'),Fact('lays eggs'))))
    def bird(self):
        self.declare('bird')
        
    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark spots'))
    def monkey(self):
        self.declare(Fact(animal='monkey'))

    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark stripes'))
    def tiger(self):
        self.declare(Fact(animal='tiger'))

    @Rule(Fact('ungulate'),
          Fact('long neck'),
          Fact('long legs'),
          Fact(pattern='dark spots'))
    def giraffe(self):
        self.declare(Fact(animal='giraffe'))

    @Rule(Fact('ungulate'),
          Fact(pattern='dark stripes'))
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('bird'),
          Fact('long neck'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def straus(self):
        self.declare(Fact(animal='ostrich'))

    @Rule(Fact('bird'),
          Fact('swims'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def pinguin(self):
        self.declare(Fact(animal='pinguin'))

    @Rule(Fact('bird'),
          Fact('flies well'))
    def albatros(self):
        self.declare(Fact(animal='albatross'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Animal is {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

Kun olemme määrittäneet tietokannan, täytämme työmuistimme joillakin alkuperäisillä tosiasioilla ja kutsumme sitten `run()`-metodia suorittaaksemme päättelyn. Näet tuloksena, että uusia pääteltyjä tosiasioita lisätään työmuistiin, mukaan lukien lopullinen tosiasia eläimestä (jos olemme asettaneet kaikki alkuperäiset tosiasiat oikein).


In [15]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='red-brown'),
    Fact(pattern='dark stripes'),
    Fact('sharp teeth'),
    Fact('claws'),
    Fact('forward looking eyes'),
    Fact('gives milk')])
ex1.run()
ex1.facts

Animal is tiger


FactList([(0, InitialFact()),
          (1, Fact(color='red-brown')),
          (2, Fact(pattern='dark stripes')),
          (3, Fact('sharp teeth')),
          (4, Fact('claws')),
          (5, Fact('forward looking eyes')),
          (6, Fact('gives milk')),
          (7, Fact('mammal')),
          (8, Fact('carnivor')),
          (9, Fact(animal='tiger'))])

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Vastuuvapauslauseke**:  
Tämä asiakirja on käännetty tekoälypohjaisella käännöspalvelulla [Co-op Translator](https://github.com/Azure/co-op-translator). Pyrimme tarkkuuteen, mutta otathan huomioon, että automaattikäännöksissä saattaa esiintyä virheitä tai epätarkkuuksia. Alkuperäinen asiakirja sen alkuperäiskielellä on virallinen ja sitova lähde. Tärkeissä asioissa suositellaan ammattilaisen tekemää ihmiskäännöstä. Emme ole vastuussa tämän käännöksen käytöstä mahdollisesti aiheutuvista väärinkäsityksistä tai tulkinnoista.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
